In [1]:
import torch
import pandas as pd
from pathlib import Path
BASE_PATH=Path('data/tiny_shakespeare/')

In [2]:
df = pd.read_csv(BASE_PATH/'train.csv')
df

,text
0,"First Citizen:\nBefore we proceed any further,..."


In [3]:
plays = df.iloc[0].text
len(plays)

1003854

In [4]:
from operator import itemgetter
from pprint import pprint

chars = sorted(list(set(plays)))
vocab_size = len(chars)
print('vocab_size', vocab_size)
char_counts = sorted([(c, plays.count(c)) for c in chars], reverse=True, key=itemgetter(1))
pprint(chars, compact=1)
pprint(char_counts, compact=1)

vocab_size 65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B',
 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R',
 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h',
 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
 'y', 'z']
[(' ', 153275), ('e', 85496), ('t', 60384), ('o', 59521), ('a', 49718),
 ('h', 46390), ('s', 44820), ('r', 44271), ('n', 43817), ('i', 40984),
 ('\n', 35525), ('l', 30239), ('d', 28462), ('u', 24055), ('m', 19968),
 ('y', 18400), (',', 17706), ('w', 15756), ('f', 14305), ('c', 14142),
 ('g', 12092), ('I', 10341), ('b', 10236), ('p', 9820), (':', 9044),
 ('v', 7098), ('.', 6960), ('A', 6648), ('k', 6394), ('T', 5971), ('E', 5567),
 ("'", 5561), ('O', 4561), ('N', 4383), ('R', 4235), ('S', 4048), ('L', 3597),
 ('C', 3480), (';', 3231), ('W', 3212), ('U', 3057), ('H', 2651), ('M', 2615),
 ('B', 2418), ('G', 2189), ('?', 2171), ('D', 1941), ('!', 1914

In [5]:
# random data check
print(plays[1000: 1500])

Second Citizen:
Would you proceed especially against Caius Marcius?

All:
Against him first: he's a very dog to the commonalty.

Second Citizen:
Consider you what services he has done for his country?

First Citizen:
Very well; and could be content to give him good
report fort, but that he pays himself with being proud.

Second Citizen:
Nay, but speak not maliciously.

First Citizen:
I say unto you, what he hath done famously, he did
it to that end: though soft-conscienced men can be
content to 


In [16]:
stoi = {chars[i]:i for i in range(len(chars))}
itos = {i:chars[i] for i in range(len(chars))}
encode = lambda s: torch.tensor([stoi[i] for i in s], dtype=torch.long)
decode = lambda i: ''.join([itos[s] for s in i.tolist()])
# test
e = encode('hi')
print(e)
d = decode(e)
print(d)

tensor([46, 47])
hi


In [17]:
block_size = 8
batch_size = 4

def get_random_batches(data):
    x_pos = torch.randint(len(data) - block_size - 1, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in x_pos])
    y = torch.stack([data[i+1:i+block_size+1] for i in x_pos])
    return x, y

get_random_batches(data)

(tensor([[53, 52, 53, 59, 56, 11,  1, 39],
         [ 1, 28, 50, 39, 52, 58, 39, 45],
         [58, 46, 43, 56,  1, 58, 59, 56],
         [47, 52,  1, 58, 46, 43, 47, 56]]),
 tensor([[52, 53, 59, 56, 11,  1, 39, 52],
         [28, 50, 39, 52, 58, 39, 45, 43],
         [46, 43, 56,  1, 58, 59, 56, 52],
         [52,  1, 58, 46, 43, 47, 56,  1]]))

In [18]:
# config class
class Config(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, value)
        
    def __getattr__(self, v):
        return super().__getitem__(v)

In [52]:
from torch import nn

class CausaulAttentionModule(torch.nn.Module):
    def __init__(self, config):
        super().__init__()
        q = nn.?
        k = nn.?
        v = nn.?
        sm = nn.Softmax()
        
        
    def forward(self, x):
        # something like this in basic format
#         q = x@self.q
#         k = x@self.k
#         v = x@self.v
        
#         return self.sm(q.dot(k))*v

        # now x is 3 * n_embed i.e, (q, k, v) * n_embed
        # linear transformation with parameters

class Block(torch.nn.Module):
    def __init__(self, config):
        super().__init__()
        attn = CausaulAttentionModule(config)
        mlp = nn.Linear(config.block_size, config.block_size)
        
    def forward(self, x):
        x = x + self.attn(x)
        x = x + self.mlp(x)
        
        
class GPT(torch.nn.Module):
    def __init__(self, config):
        super().__init__()
        print(config)
        assert config.vocab_size is not None
        assert config.block_size is not None
        
        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embed),
            wpe = nn.Embedding(config.block_size, config.n_embed),
        ))
    
    def forward(self, ids):
        b, t = ids.size()
        x = self.transformer.wte(ids)
        p = torch.arange(0, t)
        p = self.transformer.wpe(p)
        # add positional embedding and inputs
        x += p
        return x

model = GPT(Config(vocab_size=vocab_size, block_size=block_size, n_embed=10))
model(torch.randint(0, 60, (4, 8), dtype=torch.long)).shape

{'vocab_size': 65, 'block_size': 8, 'n_embed': 10}


torch.Size([4, 8, 10])

In [46]:
torch.randint(0, 60, (4, 8), dtype=)

tensor([[46, 22, 51, 13, 16,  5,  6, 13],
        [ 3, 58, 10, 25, 17,  9, 11, 59],
        [25, 54, 36, 20, 39, 48, 32, 28],
        [36, 58,  5, 51, 31, 20, 31, 42]])